In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import boto3
import pandas as pd
import urllib

#setup
CREDENTIALS_FILE = './credentials.csv'
credentials = pd.read_csv(CREDENTIALS_FILE).to_dict('records')[0]
aws_access_key_id = credentials['Access key ID']
aws_secret_access_key = credentials['Secret access key']

region_name = 'us-east-1'
endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

client = boto3.client(
    'mturk',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    endpoint_url = endpoint_url,
    region_name = region_name
)


In [22]:
#create hit type
one_minute = 60 # seconds
one_hour = 60 * one_minute
one_day = 24 * one_hour

#remember to change title!!!
hit_type_response = client.create_hit_type(
    AutoApprovalDelayInSeconds = 10 * one_minute,
    AssignmentDurationInSeconds = 5 * one_minute,
    Reward = '0.05',
    Title = 'Emotion in GIF (arousal)',
    Keywords = 'emotion, GIF, detection, VAD',
    Description = 'Choose a GIF that best describes the image shown above',
    QualificationRequirements = [
      {
          'QualificationTypeId': '00000000000000000071',
          'Comparator': 'In',
          'LocaleValues': [
            {
                'Country': 'TW'
            },
            {
                'Country': 'US'
            },
          ],
          'RequiredToPreview': True,
          'ActionsGuarded': 'PreviewAndAccept'
      },
    ]
)

In [ ]:
#hit for vad
#remember to change dim and 10/100!!!

hit_type_id = hit_type_response['HITTypeId']

dim = 'a'

with open("filenames.txt.tuples", "r") as fin:
  lines = fin.readlines()
  for l in lines:
    names = l.split('	')
    names[3] = names[3].replace("\n", "")

    params =  'dim=' + urllib.parse.quote(dim) + '&amp;' 'gifA=' + urllib.parse.quote(names[0]) + '&amp;' 'gifB=' + urllib.parse.quote(names[1]) + '&amp;''gifC=' + urllib.parse.quote(names[2]) + '&amp;''gifD=' + urllib.parse.quote(names[3]) 

    question=f'''<?xml version="1.0" encoding="UTF-8"?>
        <ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
        <ExternalURL>https://doraemon.iis.sinica.edu.tw/gif-vad/test/templates/ques-10.html?{params}</ExternalURL>
        <FrameHeight>800</FrameHeight>
        </ExternalQuestion>'''

    response = client.create_hit_with_hit_type(
          HITTypeId = hit_type_id,
          MaxAssignments = 2,
          LifetimeInSeconds = 5 * one_minute,
          Question = question,
          RequesterAnnotation = dim
      )
print("done")

In [ ]:
#fetch results

#list_hits_response = client.list_hits()
#hit_id = list_hits_response['HITs'][0]
#response = client.list_assignments_for_hit(HITId=hit_id)
hits_paginator = client.get_paginator('list_hits')
assignments_paginator = client.get_paginator('list_assignments_for_hit')
ans = ""
for hits in hits_paginator.paginate():
  for hit in hits['HITs']:
    for assignments in assignments_paginator.paginate(HITId=hit['HITId']):
      for assignment in assignments['Assignments']:
        print(assignment['Answer'])